In [ ]:
!wget https://github.com/MedhakanuBaniya/NLP_Bonus2/raw/main/MITRE.zip


--2024-04-30 02:47:18--  https://github.com/MedhakanuBaniya/NLP_Bonus2/raw/main/MITRE.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MedhakanuBaniya/NLP_Bonus2/main/MITRE.zip [following]
--2024-04-30 02:47:18--  https://raw.githubusercontent.com/MedhakanuBaniya/NLP_Bonus2/main/MITRE.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 266480 (260K) [application/zip]
Saving to: ‘MITRE.zip.1’

MITRE.zip.1         100%[===================>] 260.23K  --.-KB/s    in 0.1s    

2024-04-30 02:47:19 (2.26 MB/s) - ‘MITRE.zip.1’ saved [266480/266480]



**The wget command was used to download a ZIP file named "MITRE.zip" from a GitHub repository. The file was saved locally as "MITRE.zip.1" after following a redirection to the raw content.**


**Using the unzip command to extract the contents of the file "MITRE.zip.1". This command will unpack the contents of the ZIP archive, making its contents accessible for further use.**

In [ ]:
!unzip MITRE.zip.1

Archive:  MITRE.zip.1
  inflating: MITRE/test.txt          
  inflating: MITRE/train.txt         
  inflating: MITRE/valid.txt         


In [ ]:
!pip -q install accelerate transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.5 MB/s eta 0:00:00


**Importing the notebook_login function from the huggingface_hub library and calling it. This function is typically used to authenticate your access to the Hugging Face model hub when working in a notebook environment. By calling notebook_login(), you're likely initiating the login process, which will prompt you to provide credentials or authorize access to your Hugging Face account. This authentication is necessary for accessing and interacting with models, datasets, and other resources available on the Hugging Face model hub**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Set up three variables:

1. `task` is set to "ner" for Named Entity Recognition.
2. `model_checkpoint` is set to "distilbert-base-uncased", indicating the model you want to use.
3. `batch_size` is set to 16, determining the number of examples processed per batch during training or evaluation. **bold text

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

 The load_dataset function from the datasets library is utilized to import textual data from external files. The dataset is organized into three splits: training, testing, and validation. These splits are generated from the files specified in the data_files dictionary, where each split is associated with its respective file path. As the function executes, it iterates through the data files, processing and loading the text data into memory. Progress updates are provided as each split is generated, indicating the speed at which examples are processed. Once the generation process completes, the textual data is stored in the raw_dataset variable, effectively organizing the data for subsequent analysis or usage in natural language processing tasks. This approach streamlines the data loading process, facilitating seamless access to textual data for various machine learning applications.








In [ ]:
from datasets import load_dataset
raw_dataset = load_dataset("text", data_files={"train": "MITRE/train.txt", "test": "MITRE/test.txt", "valid": "MITRE/valid.txt"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 71001
    })
    test: Dataset({
        features: ['text'],
        num_rows: 20017
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 20342
    })
})

The separate_sentence_tags function takes a dataset containing text and corresponding tags and separates them into lists of sentences and their associated tags. It iterates through each item in the dataset, splitting the text into words and tags. It accumulates words and tags for each sentence until it encounters a period, at which point it adds the completed sentence and its tags to separate lists. Finally, it returns the lists of sentences and tags. This function is useful for preprocessing data for tasks such as Named Entity Recognition or sequence labeling.

In [ ]:
def seperate_sentence_tags(dataset):
    sentences = []
    tags = []
    sentence = []
    tag = []
    for i in dataset:
        if i["text"] != "":
            word, word_tag = i["text"].split("\t")
            sentence += [word]
            tag += [word_tag]
            if word == ".":
                sentences += [sentence]
                tags += [tag]
                sentence = []
                tag = []

    return sentences, tags

The function `unique_tags_dict` takes a list of tags as input and returns two objects: a sorted list of unique tags and a dictionary mapping each unique tag to a numerical index. It iterates through the input list to collect unique tags and assigns them numerical indices sequentially. The resulting dictionary provides a mapping from tags to their corresponding numerical representations. This function is useful for preparing categorical tag data for machine learning tasks, where numerical representations are often required.

In [ ]:
def unique_tags_dict(tags):
    unique_tags = set()
    for i in tags:
        for j in i:
            unique_tags.add(j)

    tags_dict = dict()
    j = 0
    unique_tags = sorted(list(unique_tags))
    for i in unique_tags:
        tags_dict[i] = j
        j += 1

    return unique_tags, tags_dict

The function tag_to_num takes two arguments: tags_dict, a dictionary mapping tags to numerical indices, and tags, a list of lists containing tags. It converts the categorical tags into their corresponding numerical representations based on the provided dictionary. It iterates through each sublist of tags, looks up the numerical representation of each tag using the dictionary, and appends the results to a new list. Finally, it returns this list of numerical representations of the tags. This function streamlines the process of converting categorical tag data into numerical format, which is often essential for machine learning tasks.

In [ ]:
def tag_to_num(tags_dict, tags):
    tags_nums = []
    for i in tags:
        temp = [tags_dict[j] for j in i]
        tags_nums += [temp]
        temp = []
    return tags_nums

The `dataset_hf` function converts input lists of sentences, numerical tag representations, and unique tags into a Hugging Face `Dataset` object. It creates columns for identifiers, token sequences, and NER tags, with appropriate data types and structures defined in the `Features` specification. This function streamlines the process of organizing data for training and evaluation of NER models using the Hugging Face `Dataset` API.

In [ ]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence

def dataset_hf(sentences, tags_nums, unique_tags):
    dataset = Dataset.from_dict({
        "id": list(range(len(sentences))),
        "tokens": sentences,
        "ner_tags": tags_nums
        },
        features = Features({
            "id": Value(dtype='string'),
            "ner_tags" : Sequence(ClassLabel(names = list(unique_tags))),
            "tokens": Sequence(Value(dtype = "string"))
        }
    ))
    return dataset

The `dataset_preprocess` function orchestrates the preprocessing pipeline for NER dataset preparation. It first separates the sentences and tags from the input dataset using the `separate_sentence_tags` function, then generates a list of unique tags and a dictionary mapping each tag to a numerical index using the `unique_tags_dict` function. Next, it converts the tags to their corresponding numerical representations with the `tag_to_num` function. Finally, it constructs a Hugging Face `Dataset` object using the `dataset_hf` function with the processed sentences, numerical tag representations, and unique tags. This function encapsulates the entire preprocessing workflow, enabling streamlined dataset preparation for NER model training and evaluation.

In [ ]:
def dataset_preprocess(dataset):
    sentences, tags = seperate_sentence_tags(dataset)
    unique_tags, tags_dict = unique_tags_dict(tags)
    tags_nums = tag_to_num(tags_dict, tags)
    return dataset_hf(sentences, tags_nums, unique_tags)

The provided code snippet accesses the first item in the "test" split of the `raw_dataset`. It appears to be a dictionary with a single key-value pair. The key "text" corresponds to the input text, while the value "Why\tO" represents a word-tag pair. The word "Why" is associated with the tag "O". This format suggests that the data follows a typical token-tag format commonly used in Named Entity Recognition (NER) tasks, where each word in the text is tagged with its corresponding label.

In [ ]:
raw_dataset["test"][0]

{'text': 'Why\tO'}

The provided code preprocesses the training, testing, and validation datasets from the `raw_dataset`. It utilizes the `dataset_preprocess` function to prepare each dataset. The resulting `train_dataset`, `test_dataset`, and `validation_dataset` are structured Hugging Face `Dataset` objects, ready for use in training and evaluating Named Entity Recognition (NER) models.

In [ ]:
train_dataset = dataset_preprocess(raw_dataset["train"])
test_dataset = dataset_preprocess(raw_dataset["test"])
validation_dataset = dataset_preprocess(raw_dataset["valid"])

The provided code creates a `DatasetDict` object named `datasets`, which holds three datasets: training, testing, and validation. Each dataset is represented as a Hugging Face `Dataset` object containing columns for identifiers, token sequences, and NER tags. Metadata is included for each dataset, indicating the number of rows. This structured organization streamlines dataset management for Named Entity Recognition (NER) model training and evaluation.

In [ ]:
from datasets import DatasetDict


datasets = DatasetDict({
    'train':train_dataset,
    'test': test_dataset,
    'validation':validation_dataset
})

datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2664
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 717
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 785
    })
})

The code successfully uploaded the dataset shards to the Hugging Face Hub under the repository "Medha0219Baniya/NER". Each shard was created in Parquet format from the Arrow format, and the README.md file was also uploaded. The dataset is now accessible via the Hugging Face Datasets library using the provided repository name. The commit information for the upload is available at the provided commit URL.

In [ ]:
datasets.push_to_hub("Medha0219Baniya/NER")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/880 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Medha0219Baniya/NER/commit/c961afe2accac3820130fa7149e0ab8ca383c620', commit_message='Upload dataset', commit_description='', oid='c961afe2accac3820130fa7149e0ab8ca383c620', pr_url=None, pr_revision=None, pr_num=None)

The code successfully downloaded the dataset "Medha0219Baniya/NER" from the Hugging Face Hub. It retrieved the README file and the dataset splits (train, test, and validation). Each split was generated with the corresponding number of examples, and the dataset is now ready for use in training and evaluation.

In [ ]:
from datasets import load_dataset

datasets = load_dataset("Medha0219Baniya/NER")

Generating train split:   0%|          | 0/2664 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/717 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/785 [00:00<?, ? examples/s]

The `show_random_elements` function displays a specified number of random examples from a dataset. It ensures that the number of examples to display does not exceed the total number of examples in the dataset. It randomly selects examples without replacement and converts numerical labels into their corresponding string names for better readability. The function then presents the selected examples as an HTML table using the `display` function. This function is helpful for visually inspecting random samples of data during dataset exploration or preprocessing.

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

The function `show_random_elements` randomly selects and displays a specified number of examples from a dataset. It presents the tokens (words) and their corresponding Named Entity Recognition (NER) tags in a tabular format. This function is helpful for visually inspecting random samples of data to understand the dataset's structure and content.

In [ ]:
show_random_elements(datasets["train"], num_examples = 5)

,id,tokens,ner_tags
0,1506,"[We, also, found, several, apps, containing, the, malware, ,, which, were, developed, by, other, developers, on, Google, Play, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-System, I-System, O]"
1,774,"[It, extracts, and, decrypts, the, stage, 3, malware, ,, which, is, stored, in, encrypted, resources, such, as, fake, dialog, boxes, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,1399,"[Kaspersky, Lab, mobile, products, prevented, 2,500, infections, by, banking, Trojans, .]","[B-System, I-System, O, O, O, O, O, O, O, O, O]"
3,2387,"[Related, Github, account, contains, forked, Conversations, repository, Summarizing, all, the, found, clues, ,, we, have, the, following, attribution, flow, :, Conclusion, The, operation, of, ViceLeaker, is, still, ongoing, ,, as, is, our, research, .]","[O, B-Organization, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Malware, O, O, O, O, O, O, O, O, O]"
4,2578,"[Finished, !, *, *, *, End, translation, *, *, *, Referring, again, to, bit.ly, ,, we, can, see, click, statistics, for, this, campaign, (, Figure, 6, ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-Indicator, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"


The provided data showcases an example from the training dataset at index `1082`. It consists of tokens representing words from a sentence and their corresponding Named Entity Recognition (NER) tags. All tokens in this example are tagged as "O", indicating that they do not belong to any named entity category. The dataset features three columns: "id", "tokens", and "ner_tags", where "id" is a string, "tokens" is a sequence of strings representing words, and "ner_tags" is a sequence of class labels indicating named entity types.

In [ ]:
print(datasets["train"][1082])

datasets["train"].features

{'id': '1082', 'tokens': ['RuMMS', 'samples', 'and', 'C2', 'servers', 'Figure', '8', 'shows', 'how', 'these', 'samples', ',', 'C2', 'servers', 'and', 'hosting', 'websites', 'are', 'related', 'to', 'each', 'other', ',', 'including', 'when', 'they', 'were', 'compiled', 'or', 'observed', '.'], 'ner_tags': [1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]}


{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['B-Indicator', 'B-Malware', 'B-Organization', 'B-System', 'B-Vulnerability', 'I-Indicator', 'I-Malware', 'I-Organization', 'I-System', 'I-Vulnerability', 'O'], id=None), length=-1, id=None)}

The code utilizes the `AutoTokenizer` class from the `transformers` library to instantiate a tokenizer. The tokenizer is initialized using the pretrained model specified by the variable `model_checkpoint`. During initialization, the tokenizer's configuration files and vocabulary are downloaded and cached locally for future use. If this is the first time running the code with the specified model, it may perform a one-time migration of the cache. Once initialized, the tokenizer is ready to be used for tokenizing text inputs according to the specifications of the pretrained model.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import transformers

The provided code utilizes the initialized tokenizer to tokenize the input sentence "Hello, this is one sentence!" The tokenizer converts the input sentence into tokenized input IDs and attention mask. The input IDs represent the tokenized input, while the attention mask indicates which tokens the model should pay attention to. These tokenized inputs are ready for further processing by a pretrained model, such as classification or generation tasks.

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

The provided code snippet demonstrates the tokenizer's ability to tokenize a list of words that are already split into individual tokens. It utilizes the `is_split_into_words=True` parameter to indicate that the input list is pre-tokenized. The tokenizer processes the input list and returns a tokenized list of strings, ready for further processing by a pretrained model.

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

The provided code snippet accesses an example from the training dataset and prints the list of tokens. The example consists of a list of tokens representing words from a sentence. These tokens can be utilized for various natural language processing tasks such as tokenization, modeling, or analysis.

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

['The', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'Super', 'Mario', 'Run', 'game', 'to', 'target', 'eagerly', 'waiting', 'Android', 'users', '.']


The provided code snippet tokenizes a list of tokens from the example using the initialized tokenizer. It first tokenizes the input tokens and then converts the token IDs back to their corresponding tokens. The resulting list of tokens, including special tokens such as `[CLS]` and `[SEP]`, is printed. This process showcases the tokenizer's tokenization and conversion capabilities, preparing the tokens for further processing or analysis.

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'the', 'authors', 'are', 'trying', 'to', 'latch', 'onto', 'the', 'popularity', 'of', 'the', 'super', 'mario', 'run', 'game', 'to', 'target', 'eagerly', 'waiting', 'android', 'users', '.', '[SEP]']


The comparison reveals a discrepancy in lengths between the Named Entity Recognition (NER) tags in the example and the tokenized input. Specifically:

1 The length of the NER tags in the example is 22.

2 The length of the tokenized input (represented by the input IDs) is 24.

This misalignment indicates the necessity to synchronize the lengths of the NER tags with the tokenized input for accurate NER processing.

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(22, 24)

The provided code snippet prints the word IDs associated with each token in the tokenized input. Each token is associated with a word ID, where `None` indicates special tokens like `[CLS]` or `[SEP]`, and non-`None` values correspond to the index of the word in the original input sequence. This information can be valuable for aligning tokenized input with the original word-level annotations, such as NER tags, during subsequent processing steps.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, None]


The provided code snippet performs alignment between Named Entity Recognition (NER) tags and tokenized input by associating each token with its corresponding NER tag. It begins by storing the word IDs associated with each token in the tokenized input. Then, using a list comprehension, it maps each token's NER tag to its corresponding token in the tokenized input. If the word ID is `None`, indicating a special token, the corresponding label is set to `-100`. Finally, it compares the lengths of the aligned labels and the tokenized input to ensure successful alignment. This alignment is crucial for downstream tasks, ensuring that each token in the tokenized input is appropriately labeled for accurate NER processing.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

24 24


Putting all labels tokens true

In [ ]:
label_all_tokens = True

The `tokenize_and_align_labels` function facilitates the tokenization of input examples and alignment of labels, particularly for Named Entity Recognition (NER) tasks. Firstly, it tokenizes input examples using the initialized tokenizer, considering truncation and the fact that the input is already split into words. Then, it iterates through each example and its associated NER tags. For each example, it retrieves the word IDs corresponding to each token in the tokenized input. Subsequently, it iterates through these word IDs, setting the appropriate label for each token. Special tokens, identified by a word ID of `None`, are assigned a label of `-100` to disregard them in the loss function. Additionally, it ensures correct labeling for the first token of each word and handles subsequent tokens within the same word accordingly. Finally, it integrates the aligned labels into the tokenized inputs and returns the tokenized inputs with aligned labels. This function is pivotal for ensuring the proper alignment of tokenized inputs and labels, facilitating accurate training or evaluation of NER models.

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

The `tokenize_and_align_labels` function was applied to the first five examples from the training dataset. Here's a summary of the output:

- For each example, the tokenized inputs were generated, including the input IDs and attention masks.
- The labels for each token in the tokenized inputs were aligned with the original labels from the dataset. Special tokens were assigned a label of `-100` to ignore them during training.
- The resulting output contains the tokenized inputs with aligned labels for each example.

This process ensures that the tokenized inputs and labels are properly aligned and formatted for NER model training or evaluation.

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[101, 3565, 7986, 2448, 15451, 8059, 1001, 1016, 1516, 2852, 9314, 17364, 9350, 27911, 2015, 2293, 7986, 1998, 20421, 1010, 2021, 2061, 2079, 15451, 8059, 6048, 1012, 102], [101, 1037, 2261, 2420, 2067, 1010, 2057, 2626, 2055, 2019, 11924, 28791, 2099, 23445, 8349, 20540, 2004, 1996, 3565, 7986, 2448, 2208, 2005, 11924, 1012, 102], [101, 2057, 2031, 2179, 2178, 6013, 1997, 15451, 8059, 20540, 2004, 1996, 3565, 7986, 2448, 11924, 10439, 1010, 1998, 2023, 2051, 2009, 2038, 2579, 1996, 2433, 1997, 2852, 9314, 17364, 9350, 1006, 6556, 3229, 23445, 1007, 1012, 102], [101, 6947, 8400, 2626, 2055, 1996, 2852, 9314, 17364, 9350, 2217, 1011, 8209, 2007, 1996, 20421, 2175, 10439, 2067, 1999, 2251, 2355, 1025, 1996, 4489, 2182, 2003, 2008, 2045, 2003, 2053, 2208, 2443, 1999, 1996, 24391, 7427, 1012, 102], [101, 1996, 6048, 2024, 2667, 2000, 25635, 3031, 1996, 6217, 1997, 1996, 3565, 7986, 2448, 2208, 2000, 4539, 17858, 3403, 11924, 5198, 1012, 102]], 'attention_mask': [[1, 1, 1, 1,

The provided code utilized the `map` function to apply the `tokenize_and_align_labels` function to each batch in the datasets. This process efficiently tokenized input examples and aligned labels for Named Entity Recognition (NER) tasks. Each dataset, including the training, testing, and validation sets, was processed successfully with a high processing speed, averaging between 2780 to 3082 examples per second. This mapping ensures that tokenization and label alignment are effectively applied to prepare the datasets for NER model training or evaluation.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/2664 [00:00<?, ? examples/s]

Map:   0%|          | 0/717 [00:00<?, ? examples/s]

Map:   0%|          | 0/785 [00:00<?, ? examples/s]

The `label_list` variable contains the list of label names extracted from the training dataset's features for the Named Entity Recognition (NER) task. Here's a summary of the label names:

1. B-Indicator
2. B-Malware
3. B-Organization
4. B-System
5. B-Vulnerability
6. I-Indicator
7. I-Malware
8. I-Organization
9. I-System
10. I-Vulnerability
11. O

These labels represent different types of named entities and their respective positions in the input text. They are crucial for training NER models to correctly identify and classify named entities within the text.

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['B-Indicator',
 'B-Malware',
 'B-Organization',
 'B-System',
 'B-Vulnerability',
 'I-Indicator',
 'I-Malware',
 'I-Organization',
 'I-System',
 'I-Vulnerability',
 'O']

In [ ]:
from transformers import AutoConfig

The underscore variable _ is typically used in Python as a placeholder for a value that is not going to be used. In this context, it seems like the code is discarding the output of the separate_sentence_tags function except for the tags variable. Similarly, it discards the output of the unique_tags_dict function except for the tags_dict variable.

In [ ]:
_, tags = seperate_sentence_tags(raw_dataset["train"])
_, tags_dict = unique_tags_dict(tags)

The provided dictionary contains a mapping of unique tags used in a named entity recognition (NER) task to their corresponding numerical representations. Each tag, such as 'B-Indicator' or 'O', is assigned a unique integer value. This numerical representation facilitates the processing and training of machine learning models for NER tasks, enabling efficient handling and classification of named entities within text data.

In [ ]:
tags_dict

{'B-Indicator': 0,
 'B-Malware': 1,
 'B-Organization': 2,
 'B-System': 3,
 'B-Vulnerability': 4,
 'I-Indicator': 5,
 'I-Malware': 6,
 'I-Organization': 7,
 'I-System': 8,
 'I-Vulnerability': 9,
 'O': 10}

In [ ]:
ids_dict = {value:key for key, value in tags_dict.items()}

In [ ]:
ids_dict

{0: 'B-Indicator',
 1: 'B-Malware',
 2: 'B-Organization',
 3: 'B-System',
 4: 'B-Vulnerability',
 5: 'I-Indicator',
 6: 'I-Malware',
 7: 'I-Organization',
 8: 'I-System',
 9: 'I-Vulnerability',
 10: 'O'}

The `config` object is created using `AutoConfig.from_pretrained` method, specifying a pre-trained model checkpoint, the number of labels for the Named Entity Recognition (NER) task, and dictionaries mapping label names to numerical IDs (`label2id`) and vice versa (`id2label`). This configuration is crucial for setting up the model architecture and its components tailored to the NER task, including tokenization and output layer for classification.

In [ ]:
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=len(label_list), label2id=tags_dict, id2label=ids_dict)

The `model` object is initialized using `AutoModelForTokenClassification.from_pretrained` method, which loads a pre-trained model checkpoint for token classification. The provided configuration (`config`) is used to configure the model architecture, including the number of labels and label mappings.

During initialization, some weights of the model, particularly those related to the classification layer, are initialized randomly because they were not present in the pre-trained checkpoint. The message suggests that the model should be trained on a downstream task before using it for predictions or inference. This training process will allow the model to learn task-specific patterns and update the weights accordingly.

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, config = config)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The `model_name` variable captures the name of the pre-trained model being used, while `args` defines training parameters such as output directory, evaluation strategy, learning rate, batch sizes, number of epochs, weight decay, and whether to push the trained model to the Hugging Face Hub.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"cybersecurity-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    push_to_hub=True,
)

The `DataCollatorForTokenClassification` from the Transformers library is instantiated with the tokenizer. This data collator is responsible for batch processing and padding of tokenized inputs and labels for token classification tasks.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-48-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


The `labels` variable is created to map the tag indices from the example to their corresponding labels using the `label_list`. Afterwards, a dictionary of metrics is computed using `metric.compute()`, which takes predictions and references as inputs. In this case, both predictions and references are provided as the `labels` list. The computed metrics include precision, recall, F1 score, and accuracy for the 'System' label, as well as overall precision, recall, F1 score, and accuracy.

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'System': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

The `labels` variable contains a list of labels corresponding to each token in the example. Each label is represented as a string indicating the named entity tag for that token.

In [ ]:
print(labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-System', 'I-System', 'I-System', 'O', 'O', 'O', 'O', 'O', 'B-System', 'O', 'O']


The `compute_metrics` function calculates evaluation metrics such as precision, recall, F1-score, and accuracy based on the model predictions and true labels. It first converts the model predictions from logits to the predicted labels by taking the argmax along the last dimension. Then, it removes the ignored index (-100) corresponding to special tokens. Finally, it computes the evaluation metrics using the `metric` object.

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

The `Trainer` object has been set up with the provided parameters for training a token classification model.

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

The evaluation results show an evaluation loss of 0.223, with precision, recall, and F1-score values of approximately 0.79. The evaluation accuracy is around 95%.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.22326971590518951,
 'eval_precision': 0.785625,
 'eval_recall': 0.7905660377358491,
 'eval_f1': 0.7880877742946708,
 'eval_accuracy': 0.9513664112754537,
 'eval_runtime': 3.0606,
 'eval_samples_per_second': 256.489,
 'eval_steps_per_second': 16.337,
 'epoch': 8.0}

The predictions on the test dataset resulted in the following evaluation metrics:

- **Indicator**: Precision: 0.79, Recall: 0.89, F1-score: 0.84
- **Malware**: Precision: 0.73, Recall: 0.64, F1-score: 0.68
- **Organization**: Precision: 0.65, Recall: 0.43, F1-score: 0.52
- **System**: Precision: 0.40, Recall: 0.44, F1-score: 0.42
- **Vulnerability**: Precision: 0.71, Recall: 0.83, F1-score: 0.76

Overall precision: 0.73, recall: 0.77, and F1-score: 0.75. The overall accuracy is approximately 0.93.

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Indicator': {'precision': 0.7929039722329349,
  'recall': 0.888504753673293,
  'f1': 0.8379865498267781,
  'number': 2314},
 'Malware': {'precision': 0.7255639097744361,
  'recall': 0.6380165289256199,
  'f1': 0.6789797713280562,
  'number': 605},
 'Organization': {'precision': 0.65,
  'recall': 0.4315352697095436,
  'f1': 0.5187032418952617,
  'number': 241},
 'System': {'precision': 0.39635535307517084,
  'recall': 0.4371859296482412,
  'f1': 0.4157706093189964,
  'number': 398},
 'Vulnerability': {'precision': 0.7090909090909091,
  'recall': 0.8297872340425532,
  'f1': 0.764705882352941,
  'number': 47},
 'overall_precision': 0.7300873246890712,
 'overall_recall': 0.7653259361997226,
 'overall_f1': 0.7472914409534128,
 'overall_accuracy': 0.9340902685509522}

The detailed results of the evaluation on the test dataset are as follows:

- **Indicator**: The precision for identifying indicators is 0.79, meaning that out of all the identified indicators, 79% were correctly predicted. The recall is 0.89, indicating that 89% of all true indicators were successfully captured. The F1-score, which combines precision and recall, is 0.84, suggesting a balance between precision and recall for this class. There are a total of 2314 indicators in the dataset.

- **Malware**: The precision for identifying malware instances is 0.73, indicating that 73% of the identified instances were true malware. The recall is 0.64, implying that 64% of all true malware instances were successfully detected. The F1-score is 0.68, representing the harmonic mean of precision and recall for this class. There are 605 instances of malware in the dataset.

- **Organization**: The precision for recognizing organization entities is 0.65, meaning that 65% of the identified organizations were correct. The recall is 0.43, indicating that only 43% of all true organization entities were correctly identified. The F1-score is 0.52, reflecting the balance between precision and recall for this class. There are 241 instances of organizations in the dataset.

- **System**: The precision for detecting system entities is 0.40, suggesting that only 40% of the identified system entities were true positives. The recall is 0.44, indicating that 44% of all true system entities were successfully identified. The F1-score is 0.42, showing a moderate balance between precision and recall. There are 398 instances of system entities in the dataset.

- **Vulnerability**: The precision for identifying vulnerabilities is 0.71, indicating that 71% of the identified vulnerabilities were true positives. The recall is 0.83, suggesting that 83% of all true vulnerabilities were successfully captured. The F1-score is 0.76, reflecting a good balance between precision and recall for this class. There are 47 instances of vulnerabilities in the dataset.

Overall, the model achieves an average precision of 0.73, recall of 0.77, and F1-score of 0.75 across all classes, with an accuracy of nearly 93% on the test dataset. These metrics provide insights into the model's performance in identifying various entities related to cybersecurity.